In [1]:
# NOTEBOOK IMPORTS
import os, glob, warnings, pickle
import numpy as np
from shutil import copyfile, rmtree
from datetime import datetime

# IMAGE IMPORTS
from PIL import Image, TiffImagePlugin
import cv2

# GIS IMPORTS
from affine import Affine
import pandas as pd

# PLOTTING IMPORTS
import matplotlib.pyplot as plt

# CUSTOM UTILITIES
from IndexUtils import * 

Image.MAX_IMAGE_PIXELS = 933120000
warnings.filterwarnings("ignore")
initialize = False

IO directories depending on which machine. 

In [2]:
username = os.getlogin()

dnn_params = {}
dnn_params['CLNN'] = {}
dnn_params['TPNN'] = {}
dnn_params['RLNN'] = {}
dnn_params['FLNN'] = {}
dnn_params['TLNN'] = {}


if username == 'fhacesga':
    base_input_path   = r"D:\RECTDNN\Uncompress\\"
    base_output_path  = r"D:\RECTDNN\processing\\"
    ref_dir  = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data\AAA_ReferenceDatasets\\"
elif username == 'fhace':
    base_input_path   = r"C:\Users\fhace\Desktop\FIRMs\data\Uncompress\\"
    ref_dir = r"C:\Users\fhace\Desktop\FIRMs\data\AAA_ReferenceDatasets\\"
    base_output_path = r"C:\Users\fhace\Desktop\FIRMs\data\Outputs\\"
    # dnn_params['TLNN']['model_weights']    = r"C:\Users\fhace\Desktop\FIRMs\data\BBNN\curr_weights.pt"
    dnn_params['CLNN']['model_checkpoint'] = r"C:\Users\fhace\Desktop\FIRMs\data\RLNN\checkpoint_101423.pth"
    dnn_params['TPNN']['model_checkpoint'] = r"C:\Users\fhace\OneDrive - University Of Houston\AAA_RECTDNN\data\TPNN\checkpoint_091523_pyramids_2.pth"
    dnn_params['RLNN']['model_checkpoint'] = r"C:\Users\fhace\OneDrive - University Of Houston\AAA_RECTDNN\data\RLNN\checkpoint_091323.pth"
    
else:
    base_input_path   = r"D:\Desktop\FIRMsDigitizing\data\HistoricalFIRMS"
    base_output_path  = r"D:\Desktop\FIRMsDigitizing\processing"
    ref_dir  = r"C:\Users\franc\OneDrive - University Of Houston\AAA_RECTDNN\data\AAA_ReferenceDatasets\\"

if not initialize:
    initialize = init_databases(ref_dir)

Create working dir and unzip all files if needed

In [3]:
# MAKE PROCESSING DIRECTORY
datetime_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
proc_dir     = os.path.join(base_output_path, datetime_str)

# IF THERE ARE ANY ZIP FILES IN DIRECTORY, UNZIP THEM IN PLACE
if len(glob.glob(base_input_path + "*.zip*")) != 0:
    extractZipFiles(base_input_path, base_input_path)

outputs_dir  = os.path.join(proc_dir, "Outputs")
os.makedirs(outputs_dir)

Read through all the images in the directory. Some images are saved as Multi-page TIFF files. These need to be exported into individual images. We'll do that first in-place at uncompressed directory

In [4]:
remove_files = []

for filename in tqdm(glob.glob(base_input_path + "\\*.tif*")):
    tiff_file = os.path.join(base_input_path, filename)
    try:
    # Check if the file is a multi-page TIFF
        with Image.open(tiff_file) as img:
            if img.is_animated:    
                print(f"Expanding {os.path.basename(tiff_file)} into {img.n_frames}")
                for i in range(img.n_frames):
                    try:
                        img.seek(i)
                        output_filename = f"{os.path.splitext(tiff_file)[0]}_{i+1}{os.path.splitext(tiff_file)[1]}"
                        img.save(output_filename, format=img.format)
                    except:
                        print(f"Error with {tiff_file} page {i}")
                        continue
                remove_files.append(tiff_file)
    except Exception as e:
        print(f"Error opening {tiff_file}")
        print(e)
        continue

for tiff_file in remove_files:
    os.remove(tiff_file)

  0%|          | 0/1710 [00:00<?, ?it/s]

Error opening C:\Users\fhace\Desktop\FIRMs\data\Uncompress\485479B.tif
cannot identify image file 'C:\\Users\\fhace\\Desktop\\FIRMs\\data\\Uncompress\\485479B.tif'
Error opening C:\Users\fhace\Desktop\FIRMs\data\Uncompress\485479C.tif
cannot identify image file 'C:\\Users\\fhace\\Desktop\\FIRMs\\data\\Uncompress\\485479C.tif'
Error opening C:\Users\fhace\Desktop\FIRMs\data\Uncompress\485481A.tif
cannot identify image file 'C:\\Users\\fhace\\Desktop\\FIRMs\\data\\Uncompress\\485481A.tif'
Error opening C:\Users\fhace\Desktop\FIRMs\data\Uncompress\485510.tif
cannot identify image file 'C:\\Users\\fhace\\Desktop\\FIRMs\\data\\Uncompress\\485510.tif'
Error opening C:\Users\fhace\Desktop\FIRMs\data\Uncompress\485510B.tif
cannot identify image file 'C:\\Users\\fhace\\Desktop\\FIRMs\\data\\Uncompress\\485510B.tif'
Error opening C:\Users\fhace\Desktop\FIRMs\data\Uncompress\485514B.tif
cannot identify image file 'C:\\Users\\fhace\\Desktop\\FIRMs\\data\\Uncompress\\485514B.tif'
Error opening C:\U

Here we're using heuristics to identify indices within all the image files. These include:
- Files that are shorter than 12 characters
- Files that have the ```IND``` marker

We create a Pandas DataFrame with the files matching. We then add several fields as below

In [5]:
# LIST ALL IMAGES IN DIRECTORY
image_files = glob.glob(f"{base_input_path}/*")

# FILTER IMAGES USING HEURISTICS
patterns = ["IND", "_1."]
index_files = [file for pattern in patterns for file in glob.glob(base_input_path + "\\*" + pattern + "*")]
filtered_files = [file for file in image_files if len(os.path.basename(file)) < 12]
index_files.extend(filtered_files)

# CREATE DATAFRAME
index_files = pd.DataFrame(index_files, columns=["FilePath"])

# INDEX ATTRIBUTES TO BE ADDED
index_files["Basename"] = [os.path.basename(file) for file in index_files["FilePath"].to_list()]    # BASENAME
index_files["Location"] = index_files["Basename"].apply(extract_numerical_chars).astype(np.int32)   # 
index_files["GEOID"]    = index_files["Location"].apply(getGEOID)       # GET GEOID FOR EACH INDEX
index_files["geometry"] = index_files["GEOID"].apply(getGeometry)       # GET GEOMETRY FROM MATCHING GEOIDs

In [6]:
def getBBOX_coords(tile_ds : rio.DatasetReader, bbox : list) -> list:
    """
    Converts Bounding Box pixel coordinates into actual coordinates by using the input rasterio dataset

    Parameters:
        tile_ds (rasterio.Dataset): Post-ICP rasterio dataset saved with a world file.
        bbox (iterable): Bounding box coordinates in format (x_min, y_min, x_max, y_max),
                      normalized by the total image width and height.

    Returns:
        coords (list): list of raster coordinates in (x_min, y_min, x_max, y_max) format.
    """
    # x1, y1 = rio.transform.xy(tile_ds.transform, bbox[0], bbox[1]) WRONG X, Y 
    # x2, y2 = rio.transform.xy(tile_ds.transform, bbox[2], bbox[3]) WRONG X, Y
    x1, y1 = rio.transform.xy(tile_ds.transform, bbox[1], bbox[0]) 
    x2, y2 = rio.transform.xy(tile_ds.transform, bbox[3], bbox[2])
    return [x1, y1, x2, y2]

def bbox_to_coords(bbox):
    '''
    x1, y1, x2, y2 = bbox

    x_min = np.min([x1, x2])    
    x_max = np.max([x1, x2])    
    y_min = np.min([y1, y2])    
    y_max = np.max([y1, y2])    

    
    '''
    x_min, y_min, x_max, y_max = bbox

    xs = [x_min, x_max, x_min, x_max]
    ys = [y_min, y_min, y_max, y_max]
    return xs, ys# [(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max)]

# def getTileTransform(tile):

def getTileAffine(tile, baseaffine=None):

    b = baseaffine

    w, h = tile['data'].size

    x_i = [0, w, 0, w]
    y_i = [0, 0, h, h]
    x_c, y_c = bbox_to_coords(tile['coords'])

    test = pd.DataFrame()
    test["x_i"] = x_i
    test["y_i"] = y_i
    test["x_c"] = x_c
    test["y_c"] = y_c

    a = affineTransformation(x_i, y_i, x_c, y_c)

    matrix = None
    if b is not None: 
        matrix = b.flatten()[:6]
        matrix[2] = a.matrix.flatten()[2]
        matrix[5] = a.matrix.flatten()[5]
    else:
        matrix = a.matrix.flatten()[:6]
         

    return rio.Affine(*matrix)

def saveTile(fn, tile, baseaffine=None):
    transform = tile['affine'] if 'affine' in tile else getTileAffine(tile, baseaffine=baseaffine)
    image = np.asarray(tile['data'])
    epsg_code = 3857
    with rio.open(fn, 'w',
        driver='GTiff',
        height=image.shape[0],
        width=image.shape[1],
        count=1,
        dtype=image.dtype,
        crs=f'EPSG:{epsg_code}',
        transform=transform) as dst:
            dst.write(image, 1)    

def ICPtoCRSTransform(image_arry, transform_dict):
    # REVERSE Y AXIS
    rev_y_axis = np.array([[1, 0, 0],
                        [0,-1, 0],
                        [0, 0, 1]])

    # move = original_homography @ np.array([0, image_t.shape[0], 0])
    translation = np.eye(3)
    translation[1, 2] = image_arry.shape[0]

    transform_dict['translation'] = translation
    
    # adjustment =  np.linalg.inv(transform_dict['best'].copy())
    # rev_adj = adjustment.copy()
    # rev_adj[1, 1] = rev_adj[1, 1] * -1
    transform_dict['rev_adj'] = np.linalg.inv(transform_dict['best'].copy())

    transform_dict['flip'] = np.array([
    [1, 0, 0],
    [0, -1, 0],
    [0, 0, 1]
    ])
    
    # output_transform = transform_dict['initial'] @ transform_dict['translation'] @ transform_dict['rev_adj']
    output_transform = transform_dict['initial'] @ transform_dict['rev_adj'] @ transform_dict['translation']  @ transform_dict['flip']
    offsets = output_transform @ np.array([[0, 0, 1], [image_arry.shape[0], 0, 1]]).T
    offsets = offsets[:, 1] - offsets[:, 0]
    transform_dict['offsets'] = offsets

    return output_transform, transform_dict

def findTiles(image_fn, model=None, 
        model_weights=f"{data_dir}BBNN/weights042924.pt",
        creation_params=None,
        save_dir=None,
        device="cpu",
        verbose=True
        ):
        
    input_folder = os.path.dirname(os.path.abspath(image_fn))

    if creation_params is None:
        target_size = 1920
        original_shapes = []

        # COCO DATASET PARAMS
        category_labels = {
            0 : "Tile",
            1 : "County",
            2 : "Legend",
            3 : "Box"
        }

        categories=[0, 1]
    
    # Initialize model
    if model is None:
        model = ultralytics.YOLO(model_weights).to("cpu")

    model = model.to(device)
    
    results = model(image_fn, imgsz=target_size, verbose=verbose)

    if device == "cuda":
        results = results[0].cpu()
        model   = model.to("cpu")

    # GET CLASSES AND CONFIDENCES OF EACH RESULT
    classes = results[0].boxes.data.numpy()[:, -1]
    conf    = results[0].boxes.data.numpy()[:, -2]

    # GOTTA FIND CORRECT FILE BC RESIZED WERE SAVED WITH PNG EXTENSION
    basen = os.path.basename(results[0].path)[:-4]
    in_fn = glob.glob(os.path.join(input_folder,  basen + '*[!w]'))[0]
    key = findKey(basen)

    image = Image.open(in_fn)
    width, height = image.size
    im_size_arry  = np.array([width, height, width, height])

    # OUTPUT STRUCTURE
    outputs = {}

    # FOR TILES
    slice = np.logical_and(classes==1, conf > 0.92)
    for i in np.where(slice)[0]: 
        
        # GET TILE DATA
        bbox = results[0].boxes.xyxyn.numpy()[i]
        data = extract_bounded_area(image, bbox)

        bbox = bbox * im_size_arry

        # GET ID FROM TILE
        text = pytesseract.image_to_string(data, config='--psm 12 --oem 3') # -c tessedit_char_whitelist=0123456789
        word = find_word_with_key(text, key, threshold=80, verbose=False)

        if isinstance(word, list):
            word = ",".join(word)

        outputs[word] = {"bbox" : bbox, "data" : data, "text" : text} # (bbox, data)

    # FOR COUNTY - GET MOST LIKELY BOX CLASSIFIED AS COUNTY
    county_conf = conf.copy()
    county_conf[classes != 2] = 0
    slice = np.argmax(county_conf)

    bbox = results[0].boxes.xyxyn.numpy()[slice]
    outputs["county"] = {"bbox" : bbox * im_size_arry, "data" : extract_bounded_area(image, bbox)}

    if save_dir is not None:
        results[0].save(save_dir)

    return outputs, model


verbose = True

TPNN = None
RLNN = None
CLNN = None
TLNN = None

gen_dict = {}

for i, row in tqdm(index_files.iterrows(), total=index_files.shape[0]):
    
    # try:

        filename = os.path.basename(row["FilePath"])
        
        # READ FILES AND CONVERT TO GRAYSCALE
        image = cv2.imread(row["FilePath"])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image_arry = np.asarray(image)
        
        # SAVE IMAGE TO OUTPUT DIRECTORY
        output_image_fn = os.path.join(outputs_dir, filename.split(".")[0] + ".tif")
        copyfile(row["FilePath"], output_image_fn)

        # GET BOUNDARY POINTS FROM RESPECTIVE SHAPEFILE
        output = getBoundaryPoints(row, distance=100)
        if output is None:
            print(f"COULD NOT FIND SHAPEFILE FOR {filename}")
            continue
        point_boundary_gdf, shp_bounds = output
        
        # RUN IMAGES THROUGH DNNs
        classifications, TPNN  = findKeypoints(image, model=TPNN, **dnn_params['TPNN'])
        effectiveArea, RLNN    = findSquares(image, model=RLNN, **dnn_params['RLNN'])
        countyArea, CLNN       = findCounty(image, model=CLNN, **dnn_params['CLNN'])
        tiles, TLNN            = findTiles(row["FilePath"], model=TLNN, **dnn_params['TLNN'], verbose=False,
                                           save_dir=os.path.join(outputs_dir, filename.split(".")[0] + "_BBNN.tif"))

        dnn_outputs = {
            "classifications"   : classifications,
            "effectiveArea"     : effectiveArea,
            "countyArea"    : countyArea,
            "tiles"         : tiles
        }
        
        # GET COUNTY BOUNDS IDENTIFIED IN INDEX BY TLNN
        # bounds_panels = tiles["county"]["bbox"]
        bounds_panels = find_bbox(dnn_outputs['countyArea'][:, :, 1])

        # SKIP IF WE CAN'T FIND BOUNDARY IN EXISTING DATABASES
        if point_boundary_gdf is None:
            continue

        # DEFINE BOUNDARY STRUCTU
        boundaries = {
            "point_boundary_gdf"    : point_boundary_gdf,
            "shp_bounds"            : shp_bounds,
            "bounds_panels"         : bounds_panels,
        }

        # WHAT ARE WE LOOKING FOR IN EACH IDENTIFIED SQUARE?
        key = findKey(row["Basename"])    
        if key is None:
            print(f"Could not find key in {filename}")
        
        # DO ICP
        transform_dict = performICPonIndex(boundaries, dnn_outputs, debug=False, plot=False, rotation=True, shear=False, perspective=False, icp_iterations=30)

        # GET TRANSFORM FROM ICP TO CRS TRANSFORM
        output_transform, transform_dict = ICPtoCRSTransform(image_arry, transform_dict)

        # GET OUTPUT TRANSFORM INTO AFFINE AND WRITE
        output_affine = Affine(*output_transform.flatten()[:6])
        # write_world_file_from_affine(output_affine, get_world_file_path(output_image_fn))
        with rio.open(output_image_fn, 'w',
            driver='GTiff',
            height=image.shape[0], width=image.shape[1],
            count=1, dtype=image.dtype,
            crs=f'EPSG:3857',
            transform=rio.Affine(*output_transform.flatten()[:6])) as dst:
                dst.write(image, 1)   

        tiles['transform_info'] = transform_dict
        tiles['output_transform'] = output_transform
        gen_dict[filename] = tiles

        
        src = rio.open(output_image_fn)
        for iii, (k, v) in enumerate(tiles.items()):
            if k in ['transform_info', 'output_transform']:
                continue
            tiles[k]['coords'] = getBBOX_coords(src, tiles[k]['bbox'])
            saveTile(os.path.join(outputs_dir, f"{filename}_{k}.tif"), v, baseaffine=output_transform)

            '''
            try:
                if k in ['transform_info', 'output_transform']:
                    continue
                tiles[k]['coords'] = getBBOX_coords(src, tiles[k]['bbox'])
                saveTile(os.path.join(outputs_dir, f"{filename}_{k}.tif"), v, baseaffine=output_transform)
            except Exception as e:
                print("ERROR : "+str(e) + f" {k}")
                continue
            '''

with open(os.path.join(outputs_dir, "IndexCoords.pkl"), 'wb') as handle:
    pickle.dump(gen_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
 
    

  0%|          | 0/183 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
tiles.keys()

dict_keys(['48003503508', '4800350165B', None, '4800350475', '48003506258', '1480035', '4800350600', '4800350575B', '4800350295', '4800350075,48003500658', '4800350305', '4800350425B', '4800350315B', '4800350650', '4800350050', '4800350525', '4800350285', '4800350675B', '4800350310', '4800350025B', '4800350495B', '4800350320', '49003502250', '4800350275C', '4900350195', '4800350390B', '4800350150', '4800350125', '4800350375B', '4800350155B', '4800350485', '4800350100', '4800350380C', '48003501608', '4800350490B', '4800350395', '48003501908', 'county', 'transform_info', 'output_transform'])